# Exploring the Activity Database

## Input formats

nx(<list>)+mx(<list>)+...

where
n,m are positive integers
<lists> are one or more rep counts.
    
*Example*
    
1x(1,2,3) should become [1,2,3]

### Case 0:  Setup

In [5]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import re
from sqlalchemy import (create_engine, text)
import sys

sys.path.append('..')

from notes import Notes

PGURI = os.environ['PG_AZURE_URI']
PGPW = os.environ['PG_AZURE_PW']


datasets = Path('/Users/edmundlskoviak/iCloud/Data Sets/')

## Define Functions


In [2]:
def parseOperators(inStr : str):

    # Still a few of these
    inStr.replace('*','x')
    
    if inStr.count('x') == 0:
        # basic Set
        mult = 1
        set=inStr
    else:
        (mult, set) = inStr.split('x')
        
    # need to check to see if set has an operator embedded
    # previous data clensing removed the --> (ladder operator)
    if set.count('-->') == 1:
        start, stop = set.split('-->')
        start = int(re.sub('\(','',start))
        stop = int(re.sub('\)','',stop))
        if stop < start:
            step = -1
            stop -= 1
        else:
            step = 1
            stop += 1
        set = np.arange(start, stop, step)
    else:
        # otherwise, evaluate set
        try:
            set = list(eval(set))
        except TypeError as te:
            set = list(eval(set+','))
        except NameError as ne:
            return None
    return set * int(mult)

def volume(cadence : list, weight : list) -> float:
    if np.shape(cadence) == np.shape(weight):
        return np.dot(cadence, weight)
    elif np.shape(weight) == (1,):
        return np.dot(cadence, np.repeat(weight, len(cadence)))
    elif np.shape(cadence) == (1,) and np.shape(weight) != ():
        return np.dot(np.repeat(cadence, len(weight)),weight)
    else:
        return None


In [49]:
df = pd.DataFrame([['5*[5,]','[100,100,100,100,100]'], ['[3,3,3]','[50,50,50]']])
df[0] = df[0].map(lambda x: parseOperators(x))
df[1] = df[1].map(lambda x: parseOperators(x))
#df = df.assign(volume = lambda x : volume(df[0], df[1]))
df 

,0,1
0,"[5, 5, 5, 5, 5]","[100, 100, 100, 100, 100]"
1,"[3, 3, 3]","[50, 50, 50]"


# Get the data from the Azure Database

In [9]:
engine = create_engine(os.environ['PG_AZURE_URI'])
sets_qry_stmt = """
SELECT s.id
    ,date
    ,exercise_name
    ,cadence
    ,weights
    ,weight_unit
    ,notes
FROM sets s
JOIN exercises e ON e.id = s.exercise_id
ORDER BY date; 
"""
with engine.connect() as conn:
    raw_sets_df = pd.read_sql_query(text(sets_qry_stmt), conn)

In [4]:
raw_sets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             1200 non-null   int64         
 1   date           1200 non-null   datetime64[ns]
 2   exercise_name  1200 non-null   object        
 3   cadence        1200 non-null   object        
 4   weights        1200 non-null   object        
 5   weight_unit    1200 non-null   object        
 6   notes          1200 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 65.8+ KB


# This is used to Test and do data cleanup

In [10]:
notes = Notes()
for row in raw_sets_df.itertuples():
        volume_flag = True
        cadence = parseOperators(row[4])
        weight = parseOperators(row[5])
        if weight == None:
                weight = str(row[5])
                volume_flag = False
                
        print(f'Id: {row[1]}', end='')
        print(f' Cadence: {cadence} Weights: {weight}', end='')
        #print(f' shape(cadence): {np.shape(cadence)} shape(weights): {np.shape(weight)}', end='')        
        if volume_flag:
                print(f' Volume: {volume(cadence, weight)}')
        else:
                print(f' Voume: NOT CALC')
        #print(f'Notes: {notes.decode_notes(row[7])}')
    


Id: 6228 Cadence: [6, 5, 4, 3, 3] Weights: [45, 45, 50, 60, 65] Volume: 1070
Id: 6229 Cadence: [3, 4, 7, 4] Weights: [60, 65, 65, 65] Volume: 1155
Id: 6230 Cadence: [4, 6, 10, 6] Weights: [95, 100, 135, 135] Volume: 3140
Id: 6231 Cadence: [4, 6, 10, 6] Weights: [20, 20, 20, 20] Volume: 520
Id: 6232 Cadence: [6, 5, 4, 3, 3] Weights: [75, 95, 100, 105, 105] Volume: 1955
Id: 6233 Cadence: [6, 5, 4, 3, 3] Weights: [135, 145, 155, 165, 170] Volume: 3160
Id: 6234 Cadence: [3, 4, 7, 4] Weights: [135, 155, 185, 205] Volume: 3140
Id: 6235 Cadence: [4, 6, 10, 6] Weights: [95, 95, 95, 95] Volume: 2470
Id: 6236 Cadence: [6, 5, 4, 3, 3] Weights: [185, 225, 250, 270, 275] Volume: 4870
Id: 6237 Cadence: [3, 4, 7, 4] Weights: [55, 55, 55, 55] Volume: 990
Id: 6238 Cadence: [4, 6, 10, 6] Weights: [10, 10, 10, 10] Volume: 260
Id: 6239 Cadence: [4, 6, 10, 6] Weights: [22, 22, 22, 22] Volume: 572
Id: 6240 Cadence: [1, 3, 1, 1, 1] Weights: [60, 60, 60, 70, 70] Volume: 440
Id: 6241 Cadence: [1, 1] Weights: [

# Expore getting metrics for a particular exercise

In [34]:
exercise = 'Barbell Squat'
#ex_df = raw_sets_df[raw_sets_df['exercise_name'] ==  exercise]
ex_df = raw_sets_df[raw_sets_df['exercise_name'] ==  exercise]
ex_df = ex_df.assign(vol=volume(parseOperators(ex_df['weights']), parseOperators(ex_df['cadence'])))

/var/folders/hb/rg77cvs94cj4d3ry4mdmgn6h0000gn/T/ipykernel_15895/1818003610.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. ser.count(level=1) should use ser.groupby(level=1).count().
  if inStr.count('x') == 0:


ValueError: Series.count level is only valid with a MultiIndex

In [35]:
ex_df

,id,date,exercise_name,cadence,weights,weight_unit,notes
70,6298,2021-07-25 00:00:00,Barbell Squat,"1,3,3,1,2","[155,165,175,190,200]",lb,
75,6303,2021-07-26 00:00:00,Barbell Squat,"1,3,4,1,3,4,2,4",185,lb,
98,6321,2021-11-14 00:00:00,Barbell Squat,"[5,5,5,5,5]","[190,190,190,195,195]",lb,
129,6357,2021-12-01 00:00:00,Barbell Squat,"[5,5,5,5,5]",185,lb,
343,6569,2022-02-12 00:00:00,Barbell Squat,"[5,5,5,5,5]","[185,195,195,200,200]",lb,|4875-224.6
505,6733,2022-03-29 00:00:00,Barbell Squat,"[4,4,4,4]","[135,150,170,185]",lb,_a|2560-180.9
529,6761,2022-04-04 00:00:00,Barbell Squat,"[10,10,10,10]","[135,155,165,175]",lb,_a|6300-200.6
572,6803,2022-04-15 00:00:00,Barbell Squat,"(12,10,7,5,3)","[170,175,175,195,205]",lb,|6695-215.4
584,6813,2022-04-18 00:00:00,Barbell Squat,"[8,8,8]","[180,180,190]",lb,_a
602,6834,2022-04-25 00:00:00,Barbell Squat,"(8,6,6)","[185,200,215]",lb,_a


In [15]:
exercise = 'Bench Press'
with open( datasets / 'benchpress.dat', 'w') as fp:
     fp.write('date\tweights\tcadence\tvolume\tweight_unit\tnotes\n')
     for row in raw_sets_df[raw_sets_df['exercise_name'] == exercise].itertuples():
          volume_flag = True
          date = row[2]
          cadence = parseOperators(row[4])
          weight = parseOperators(row[5])
          if weight == None:
               weight = str(row[5])
               volume_flag = False
          if volume_flag:
               v = volume(cadence, weight)
          else:
               v = 'NaN'
          fp.write(f'{date}\t{cadence}\t{weight}\t{v}\t{row[6]}\t{row[7]}\n')



In [ ]:



     
     



d = {'date' : raw_sets_df[raw_sets_df['exercise_name'] == 'Bench Press']['date'],
     'weights' : raw_sets_df[raw_sets_df['exercise_name'] == 'Bench Press']['weights'].transform(lambda x : parseOperators(x)),
     'cadence' : raw_sets_df[raw_sets_df['exercise_name'] == 'Bench Press']['cadence'].transform(lambda x : parseOperators(x)),
     'units' : raw_sets_df[raw_sets_df['exercise_name'] == 'Bench Press']['weight_unit'],
     'notes' : raw_sets_df[raw_sets_df['exercise_name'] == 'Bench Press']['notes']
}

proc_df = pd.DataFrame(data = d)
proc_df.assign()

In [192]:
proc_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 33 to 1175
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     51 non-null     datetime64[ns]
 1   weights  51 non-null     object        
 2   cadence  51 non-null     object        
 3   units    51 non-null     object        
 4   notes    51 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.4+ KB


In [180]:
try:
    a = np.array([155,135])
    b = np.array([1.55,1.35])
    print(np.arccos(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))))
except TypeError as te:
    print(None)
except ValueError as ve:
    print(ve)

0.0


In [47]:
df = pd.read_csv( datasets / 'sets.csv')

### Case 1:  nx(<list>)

In [ ]:
for cadence in raw_sets_df['cadence']:
    listFinal = []
    for token in tokenize(cadence):
        listFinal.extend(parseOperators(token))
    print(f"In: {cadence}\t Out: {listFinal}")

In [ ]:
for weight in raw_sets_df['weights']:
    listFinal = []
    for token in tokenize(weight):
        listFinal.extend(parseOperators(token))
    print(f"In: {weight}\tOut: {listFinal}")